In [1]:
## Get some random module
!pip install nbastats
import nbastats.nbastats as nbastats
!pip install py-goldsberry
import goldsberry

In [2]:
## Import a bunch of modules that may be helpful
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
import requests

In [326]:
## Saving what we merged, checking the shape to see how many shots we didn't have
joined_shots.to_csv("joined_shots_2014.csv")
print joined_shots.shape,all_shots.shape,all_chart.shape

(200532, 40) (203590, 21) (205550, 22)


In [9]:
## function for getting the shot log of a player given his ID and the year
def shot_log(year,player_id):
    api_param = {'DateFrom' : '',
                    'DateTo' : '',
                    'GameSegment': '',
                    'LastNGames': 0,
                    'LeagueID': "00",
                    'Location': '',
                    'Month': 0,
                    'OpponentTeamID': 0,
                    'Outcome': '',
                    'Period': 0,
                    'PlayerID': player_id,
                    'Season': year,
                    'SeasonSegment': '',
                    'SeasonType': "Regular Season",
                    'TeamID': 0,
                    'VsConference':'',
                    'VsDivision': ''
                               }
    pull=requests.get("http://stats.nba.com/stats/playerdashptshotlog?",params=api_param)
    headers = pull.json()['resultSets'][0]['headers']
    values = pull.json()['resultSets'][0]['rowSet']
    huh=pd.DataFrame([dict(zip(headers, value)) for value in values])
    return huh


In [366]:
# get players in season a
def players(a):
    url = "http://stats.nba.com/stats/commonallplayers?"
    api_param = {'IsOnlyCurrentSeason':"0",
                          'LeagueID': '00',
                          'Season': '2015-16'}
    pull = requests.get(url, params=api_param)
    _headers = pull.json()['resultSets'][0]['headers']
    _values = pull.json()['resultSets'][0]['rowSet']
    season=pd.DataFrame([dict(zip(_headers, value)) for value in _values])
    season = season.query("FROM_YEAR <= @a and TO_YEAR >= @a")
    return season

In [ ]:
players2014=players('2014')
players2013=players('2013')

In [293]:
#Get shot chart and shot log for all players in 2014
#Don't run, already been saved
all_shots= pd.DataFrame()
all_chart = pd.DataFrame()
for index, row in players2014.iterrows():
    chart = nbastats.ShotChart(row["PERSON_ID"],season="2014-15")
    all_chart=all_chart.append(chart.shotchart())
    shots = shot_log("2014-15",row["PERSON_ID"])
    shots['PLAYER_ID']=row["PERSON_ID"]
    all_shots = all_shots.append(shots)
all_shots.to_csv("all_shots_2014.csv")
all_chart.to_csv("all_chart_2014.csv")

In [295]:
#Get shot chart and shot log for all players in 2013
#Don't run, already been saved
all_shots= pd.DataFrame()
all_chart = pd.DataFrame()
for index, row in season2013.iterrows():
    shots = shot_log("2013-14",row["PERSON_ID"])
    shots['PLAYER_ID']=row["PERSON_ID"]
    all_shots = all_shots.append(shots)
    chart = nbastats.ShotChart(row["PERSON_ID"],season="2013-14")
    all_chart=all_chart.append(chart.shotchart())
all_shots.to_csv("all_shots_2013.csv")
all_chart.to_csv("all_chart_2013.csv")

In [5]:
## Merging shot chart data with player tracking for 2014
## DO NOT RUN... takes a while
all_shots = pd.read_csv("all_shots_2013.csv")
all_chart = pd.read_csv("all_chart_2013.csv")
players=set(all_shots['PLAYER_ID'])
joined_shots=pd.DataFrame()
joined_shots_1=pd.DataFrame()
for player in players:
    shots = all_shots.loc[all_shots.PLAYER_ID==player].reset_index(drop=True)
    chart = all_chart.loc[all_chart.PLAYER_ID==player].reset_index(drop=True)
    split_time = shots.GAME_CLOCK.str.split(':')
    col_names=[u'CLOSEST_DEFENDER', u'CLOSEST_DEFENDER_PLAYER_ID', u'CLOSE_DEF_DIST', 
               u'DRIBBLES', u'FGM', u'FINAL_MARGIN', u'GAME_CLOCK', u'GAME_ID', u'LOCATION', 
               u'MATCHUP', u'PERIOD', u'PTS', u'PTS_TYPE', u'SHOT_CLOCK', u'SHOT_DIST', 
               u'SHOT_NUMBER', u'SHOT_RESULT', u'TOUCH_TIME', u'W',u'GRID_TYPE',u'GAME_EVENT_ID',
               u'PLAYER_ID', u'PLAYER_NAME', u'TEAM_ID',u'MINUTES_REMAINING', u'SECONDS_REMAINING', 
               u'EVENT_TYPE', u'ACTION_TYPE', u'SHOT_TYPE', u'SHOT_ZONE_BASIC', 
               u'SHOT_ZONE_AREA', u'SHOT_ZONE_RANGE', u'SHOT_DISTANCE', u'LOC_X', u'LOC_Y', 
               u'SHOT_ATTEMPTED_FLAG', u'SHOT_MADE_FLAG']
    time = pd.DataFrame(shots.GAME_CLOCK.str.split(':').tolist(), columns="MIN SECONDS".split())
    shots['MIN'] = time['MIN']
    shots['SECONDS'] = time['SECONDS']
    shots['TIME']=60*shots.MIN.astype(int)+shots.SECONDS.astype(int)
    chart['TIME']=60*chart.MINUTES_REMAINING+chart.SECONDS_REMAINING
    games = set(chart["GAME_ID"])
    for game in games:
        game_shots_chart=chart.loc[chart.GAME_ID==game].reset_index(drop=True)
        game_shots_log = shots.loc[shots.GAME_ID==game].reset_index(drop=True)
        if game_shots_chart.shape[0]==game_shots_log.shape[0]:
            huh = pd.concat([game_shots_chart,game_shots_log],axis=1)
            huh=huh[col_names]
            joined_shots=joined_shots.append(huh)
        else:
            quarters=set(game_shots_chart['PERIOD'])
            for quarter in quarters:
                same_period=game_shots_log.loc[(game_shots_log.PERIOD==quarter)].reset_index(drop=True)
                same_period_chart=game_shots_chart.loc[(game_shots_chart.PERIOD==quarter)].reset_index(drop=True)
                if (not same_period.empty) and (not same_period_chart.empty):
                    if same_period_chart.shape[0]==same_period.shape[0]:
                        huh = pd.concat([same_period_chart,same_period],axis=1)
                        huh=huh[col_names]
                        joined_shots=joined_shots.append(huh)
                    else:
                        for index,row in same_period_chart.iterrows():
                            same_period['DIF']=abs(same_period.TIME-row.TIME)
                            df=same_period.sort(['DIF'],ascending=True)
                            same_time=same_period.loc[[0]]
                            if same_time.DIF[0]<5:
                                a=same_period_chart.loc[[index]].reset_index(drop=True)
                                huh=pd.concat([a,same_time],axis=1)
                                huh=huh[col_names]
                                joined_shots_1=joined_shots_1.append(huh,ignore_index=True)
    print player
joined_shots

202779
200745
200746
200751
200752
200755
200756
200757
200758
200761
2399
200765
200768
200769
200770
200777
200779
200780
200782
200794
200797
2405
2406
200811
203112
200826
200839
203458
203459
101107
2199
2200
203460
2203
2207
2210
2215
2216
2222
2223
2224
2225
2229
202952
202962
202970
203471
203476
101108
203144
2365
203076
203077
203078
203079
203080
203081
203082
203083
203084
203085
203086
203087
203088
203089
203090
203091
203092
203093
203094
203095
203096
203098
203099
203100
203101
203102
203103
2400
203106
2403
203108
203109
203110
203111
2408
203113
203114
203115
203116
203117
203118
203120
203121
2419
203124
2422
203129
203130
2427
203133
2430
203135
203136
203138
203139
203141
203142
203143
2440
203145
203146
203148
2446
2449
203156
2457
203506
203183
203187
201141
201142
201143
201144
201145
201147
201148
201149
201150
201152
201155
201156
2501
201158
201160
201162
201163
201166
201167
201168
201171
201175
201177
203515
201188
201189
201196
2544
201202
2547
2548
2549


CLOSEST_DEFENDER  CLOSEST_DEFENDER_PLAYER_ID  CLOSE_DEF_DIST  DRIBBLES  FGM  FINAL_MARGIN GAME_CLOCK   GAME_ID   GAME_ID LOCATION                     MATCHUP  PERIOD  PERIOD  PTS  PTS_TYPE  SHOT_CLOCK  SHOT_DIST  SHOT_NUMBER SHOT_RESULT  TOUCH_TIME  W          GRID_TYPE  GAME_EVENT_ID  PLAYER_ID  PLAYER_ID    PLAYER_NAME     TEAM_ID  MINUTES_REMAINING  SECONDS_REMAINING   EVENT_TYPE             ACTION_TYPE       SHOT_TYPE        SHOT_ZONE_BASIC         SHOT_ZONE_AREA  \
0               Turner, Evan                      202323             1.9        17    0            10       8:09  21300165  21300165        A    NOV 20, 2013 - TOR @ PHI       2       2    0         2         9.8        7.7            1      missed        13.5  W  Shot Chart Detail            163     202779     202779  Dwight Buycks  1610612761                  8                  6  Missed Shot               Jump Shot  2PT Field Goal  In The Paint (Non-RA)              Center(C)   
1               Turner, Evan                      202323             3.2         0    1            10       9:10  21300165  21300165        A    NOV 20, 2013 - TOR @ PHI       4       4    3         3         8.1       23.5            2        made         1.4  W  Shot Chart Detail            449     202779     202779  Dwight Buycks  1610612761                  9                  7    Made Shot               Jump Shot  3PT Field Goal      Above the Break 3  Right Side Center(RC)   
2   Carter-Williams, Michael                      203487             3.5         0    0            10       8:36  21300165  21300165        A    NOV 20, 2013 - TOR @ PHI       4       4    0         3        11.0       25.7            3      missed         1.3  W  Shot Chart Detail            452     202779     202779  Dwight Buycks  1610612761                  8                 33  Missed Shot               Jump Shot  3PT Field Goal      Above the Break 3  Right Side Center(RC)   
3             Brown, Lorenzo                      203485             3.6        19    0            10       0:23  21300165  21300165        A    NOV 20, 2013 - TOR @ PHI       4       4    0         3         9.0       24.7            4      missed        14.5  W  Shot Chart Detail            575     202779     202779  Dwight Buycks  1610612761                  0                 21  Missed Shot               Jump Shot  3PT Field Goal      Above the Break 3              Center(C)   
0               Cole, Norris                      202708             3.7         0    1            -7      11:43  21300230  21300230        H  NOV 29, 2013 - TOR vs. MIA       2       2    3         3         NaN       25.2            1        made         1.0  L  Shot Chart Detail            134     202779     202779  Dwight Buycks  1610612761                 11                 41    Made Shot               Jump Shot  3PT Field Goal      Above the Break 3   Left Side Center(LC)   
1            Andersen, Chris                        2365             1.6        13    0            -7       9:36  21300230  21300230        H  NOV 29, 2013 - TOR vs. MIA       2       2    0         2         NaN        9.3            2      missed         9.3  L  Shot Chart Detail            146     202779     202779  Dwight Buycks  1610612761                  9                 34  Missed Shot               Jump Shot  2PT Field Goal              Mid-Range           Left Side(L)   
2               Cole, Norris                      202708             2.4         5    0            -7       8:21  21300230  21300230        H  NOV 29, 2013 - TOR vs. MIA       2       2    0         3         2.6       28.4            3      missed         4.5  L  Shot Chart Detail            159     202779     202779  Dwight Buycks  1610612761                  8                 18  Missed Shot               Jump Shot  3PT Field Goal      Above the Break 3   Left Side Center(LC)   
0            Jones, Terrence                      203093             3.4         6    0            -6      10:48  21

In [6]:
## Saving what we merged, checking the shape to see how many shots we didn't have
joined_shots.to_csv("joined_shots_2013.csv")
print joined_shots.shape,all_shots.shape,all_chart.shape

(200963, 40) (202689, 21) (204126, 22)


In [ ]:
##NEED TO DO THE SAME FOR 2013

In [345]:
joined_shots=pd.read_csv("joined_shots_2014.csv")
max(joined_shots.LOC_Y)
plt.scatter(joined_shots.LOC_X,joined_shots.LOC_Y)
plt.show()

In [350]:
huh=joined_shots[joined_shots.PTS_TYPE==2]
plt.scatter(huh.LOC_X,huh.LOC_Y)
plt.show()